## Import Libraries

In [ ]:
import argparse
import time
import numpy as np
from multiprocessing import Queue, Pipe
import brainflow
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds
from brainflow.data_filter import DataFilter, FilterTypes, AggOperations
from pylsl import StreamInfo, StreamOutlet, local_clock

##  Configure Board Parameters with Brainflow

In [ ]:
params = BrainFlowInputParams();
#params.ip_address = "192.168.4.1"; # for direct connect
params.ip_address = "192.168.0.101";
params.ip_port = 6677;
params.timeout = 0;
board_id = BoardIds.CYTON_DAISY_WIFI_BOARD.value;
board = BoardShim(board_id,params);
print("Sampling Rate: ", board.get_sampling_rate(board_id));
print("Device name: ", board.get_device_name(board_id));

In [ ]:
board.prepare_session()
board.config_board("~5") # set the sampling rate back to 500Hz
srate = 500

## LSL Initialization 

In [ ]:
# LSL initialization
eeg_channels = BoardShim.get_eeg_channels(board_id) # get eeg channel index
accel_channels = BoardShim.get_accel_channels(board_id) # get accel channel index
res = np.append(eeg_channels, accel_channels)
n_channels = len(res) # get the eeg channel count

# Create LSL Stream
info = StreamInfo('WeDAQ1', 'EXG', n_channels, srate, 'float32', 'WeDAQtest1')
outlet = StreamOutlet(info)

## Board Settings

### Channel Settings

# Single Channel
board.config_board("x1060001X")
board.config_board("x2161000X")
board.config_board("x3161000X")
board.config_board("x4161000X")
board.config_board("x5161000X")
board.config_board("x6161000X")
board.config_board("x7161000X")
board.config_board("x8161000X")
board.config_board("xQ161000X")
board.config_board("xW161000X")
board.config_board("xE161000X")
board.config_board("xR161000X")
board.config_board("xT161000X")
board.config_board("xY161000X")
board.config_board("xU161000X")
board.config_board("xI161000X")

## Start Streaming Data

### Configure Stream 

In [ ]:
# start stream
board.start_stream() # 450,000 Max
time.sleep(1)

### Stream to LSL

In [ ]:
SCALE_FACTOR_EEG = (4500000)/24/(2**23-1) #uV/count #You can use this in the signal processing after LSL
SCALE_FACTOR_AUX = 0.002 / (2**4) 

# read data with brainflow and send it via LSL
print("Now sending data...")
board.config_board("P"); # Unchain Mode Set
while True:
    data = board.get_board_data()[res]
    mychunk = [];
    for i in range(len(data[0])):
        mychunk.append(data[:,i].tolist())
    if len(mychunk) > 0:
        outlet.push_chunk(mychunk)
    time.sleep(1)